In [1]:
%matplotlib inline
from fastai import *
from fastai.vision import *
from fastai.callbacks.tracker import *
from fastai.vision.models.wrn import wrn_22


torch.backends.cudnn.benchmark = True

In [2]:
#from Init27 notebook, a generic training with 
#https://github.com/EricPerbos/RTX-2080Ti-Vs-GTX-1080Ti-CIFAR-100-Benchmarks/blob/master/1080Ti%20Notebook.ipynb

import functools
import traceback
def get_ref_free_exc_info():
    "Free traceback from references to locals/globals to avoid circular reference leading to gc.collect() unable to reclaim memory"
    type, val, tb = sys.exc_info()
    traceback.clear_frames(tb)
    return (type, val, tb)

def gpu_mem_restore(func):
    "Reclaim GPU RAM if CUDA out of memory happened, or execution was interrupted"
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except:
            type, val, tb = get_ref_free_exc_info() # must!
            raise type(val).with_traceback(tb) from None
    return wrapper

In [3]:
class gpu_mem_restore_ctx():
    " context manager to reclaim GPU RAM if CUDA out of memory happened, or execution was interrupted"
    def __enter__(self): return self
    def __exit__(self, exc_type, exc_val, exc_tb):
        if not exc_val: return True
        traceback.clear_frames(exc_tb)
        raise exc_type(exc_val).with_traceback(exc_tb) from None

In [4]:
torch.cuda.set_device(0)
torch.cuda.current_device()

0

In [5]:
torch.cuda.get_device_name(0)

'GeForce RTX 2060'

In [6]:
path = untar_data(URLs.CIFAR)
path

PosixPath('/home/eric/Link_fastaiV1/data/cifar10')

## ResNet 18 in FP32

In [7]:
bs = 512
np.random.seed(42)

In [8]:
ds_tfms = ([*rand_pad(4, 32), flip_lr(p=0.5)], [])
data = ImageDataBunch.from_folder(path, valid='test', ds_tfms=ds_tfms, bs=bs).normalize(cifar_stats)

In [9]:
learn = create_cnn(data, models.resnet18, metrics=accuracy)

In [10]:
with gpu_mem_restore_ctx():
    learn.fit_one_cycle(30)

epoch,train_loss,valid_loss,accuracy
1,2.290966,1.855649,0.356500
2,1.969838,1.659049,0.429000
3,1.775563,1.543919,0.458000
4,1.627276,1.444731,0.495300
5,1.502891,1.359157,0.521300
6,1.398134,1.277273,0.539500
7,1.320497,1.203179,0.564800
8,1.253077,1.139623,0.596000
9,1.194359,1.094885,0.602100
10,1.138010,1.048519,0.625100


## ResNet 18 in FP16  (Mixed-Precision)

In [11]:
bs = 512
np.random.seed(42)

In [12]:
ds_tfms = ([*rand_pad(4, 32), flip_lr(p=0.5)], [])
data = ImageDataBunch.from_folder(path, valid='test', ds_tfms=ds_tfms, bs=bs).normalize(cifar_stats)

In [13]:
learn = create_cnn(data, models.resnet18, metrics=accuracy).to_fp16()

In [14]:
with gpu_mem_restore_ctx():
    learn.fit_one_cycle(30)

epoch,train_loss,valid_loss,accuracy
1,2.291689,1.841438,0.359500
2,1.971471,1.647363,0.428400
3,1.777683,1.531595,0.464100
4,1.617684,1.436216,0.491700
5,1.505668,1.348824,0.526200
6,1.412965,1.272351,0.545100
7,1.328040,1.203399,0.570400
8,1.257717,1.144152,0.587900
9,1.191667,1.081405,0.615700
10,1.139551,1.047945,0.628500


In [15]:
# batch_size x2
bs = 1024
np.random.seed(42)

In [16]:
ds_tfms = ([*rand_pad(4, 32), flip_lr(p=0.5)], [])
data = ImageDataBunch.from_folder(path, valid='test', ds_tfms=ds_tfms, bs=bs).normalize(cifar_stats)

In [17]:
learn = create_cnn(data, models.resnet18, metrics=accuracy).to_fp16()

In [18]:
with gpu_mem_restore_ctx():
    learn.fit_one_cycle(30)

epoch,train_loss,valid_loss,accuracy
1,2.467145,1.943421,0.320400
2,2.165519,1.712972,0.409600
3,1.946670,1.585969,0.451800
4,1.779075,1.492972,0.474800
5,1.650083,1.415646,0.505200
6,1.541158,1.346980,0.521700
7,1.453888,1.282794,0.542700
8,1.376431,1.217526,0.564600
9,1.307415,1.170159,0.580200
10,1.251790,1.127531,0.598400


## WideResNet_22 in FP32
https://docs.fast.ai/vision.models.html

In [19]:
bs = 160
np.random.seed(42)

In [20]:
ds_tfms = ([*rand_pad(4, 32), flip_lr(p=0.5)], [])
data = ImageDataBunch.from_folder(path, valid='test', ds_tfms=ds_tfms, bs=bs).normalize(cifar_stats)

In [21]:
learn = Learner(data, wrn_22(), metrics=accuracy)

In [22]:
with gpu_mem_restore_ctx():
    learn.fit_one_cycle(30)

epoch,train_loss,valid_loss,accuracy
1,1.317048,1.396385,0.490500
2,1.047010,1.217235,0.578000
3,0.869487,0.899063,0.682600
4,0.734136,0.997573,0.680100
5,0.644894,0.898885,0.697300
6,0.588601,0.819666,0.729200
7,0.493651,0.719923,0.764200
8,0.465325,0.609092,0.811000
9,0.391901,0.415801,0.857200
10,0.364507,0.446335,0.853300


## WideResNet_22 in FP16 (Mixed-Precision)
https://docs.fast.ai/vision.models.html

In [23]:
bs = 160
np.random.seed(42)

In [24]:
ds_tfms = ([*rand_pad(4, 32), flip_lr(p=0.5)], [])
data = ImageDataBunch.from_folder(path, valid='test', ds_tfms=ds_tfms, bs=bs).normalize(cifar_stats)

In [25]:
learn = Learner(data, wrn_22(), metrics=accuracy).to_fp16()

In [26]:
with gpu_mem_restore_ctx():
    learn.fit_one_cycle(30)

epoch,train_loss,valid_loss,accuracy
1,1.304452,1.248219,0.555200
2,1.018844,1.593796,0.499100
3,0.860586,1.208421,0.607200
4,0.715602,0.998704,0.691000
5,0.635556,0.840220,0.724000
6,0.573549,1.839876,0.576000
7,0.515428,0.685207,0.779300
8,0.450576,0.528060,0.826600
9,0.405363,0.464943,0.850300
10,0.355685,0.398853,0.867600


In [27]:
# batch_size x2
bs = 320
np.random.seed(42)

In [28]:
ds_tfms = ([*rand_pad(4, 32), flip_lr(p=0.5)], [])
data = ImageDataBunch.from_folder(path, valid='test', ds_tfms=ds_tfms, bs=bs).normalize(cifar_stats)

In [29]:
learn = Learner(data, wrn_22(), metrics=accuracy).to_fp16()

In [30]:
with gpu_mem_restore_ctx():
    learn.fit_one_cycle(30)

epoch,train_loss,valid_loss,accuracy
1,1.553356,1.414831,0.484000
2,1.182553,1.245058,0.560600
3,0.954424,1.022316,0.638700
4,0.779197,1.286685,0.598900
5,0.682904,0.762695,0.745100
6,0.602559,1.725804,0.577300
7,0.536268,0.664288,0.781800
8,0.478985,0.760422,0.760000
9,0.427568,0.648009,0.783400
10,0.381798,0.581680,0.817200


## ResNet 34 in FP32


In [31]:
bs = 256
np.random.seed(42)

In [32]:
ds_tfms = ([*rand_pad(4, 32), flip_lr(p=0.5)], [])
data = ImageDataBunch.from_folder(path, valid='test', ds_tfms=ds_tfms, bs=bs).normalize(cifar_stats)

In [33]:
learn = create_cnn(data, models.resnet34, metrics=accuracy)

In [34]:
with gpu_mem_restore_ctx():
    learn.fit_one_cycle(30)

epoch,train_loss,valid_loss,accuracy
1,2.063694,1.744105,0.413400
2,1.764239,1.531876,0.472000
3,1.583976,1.395565,0.508800
4,1.438899,1.291697,0.538100
5,1.331417,1.191694,0.578600
6,1.211262,1.094517,0.610500
7,1.137514,1.008772,0.641700
8,1.067359,0.953097,0.658700
9,1.002107,0.904099,0.676700
10,0.962913,0.870358,0.692700


## ResNet 34 in FP16 (Mixed-Precision)

In [35]:
bs = 256
np.random.seed(42)

In [36]:
ds_tfms = ([*rand_pad(4, 32), flip_lr(p=0.5)], [])
data = ImageDataBunch.from_folder(path, valid='test', ds_tfms=ds_tfms, bs=bs).normalize(cifar_stats)

In [37]:
learn = create_cnn(data, models.resnet34, metrics=accuracy).to_fp16()

In [38]:
with gpu_mem_restore_ctx():
    learn.fit_one_cycle(30)

epoch,train_loss,valid_loss,accuracy
1,2.056200,1.725823,0.411000
2,1.773574,1.527135,0.470500
3,1.579673,1.403912,0.507000
4,1.438452,1.290940,0.543300
5,1.316929,1.190445,0.577100
6,1.223943,1.085823,0.614700
7,1.136203,1.022360,0.628300
8,1.077801,0.953528,0.659200
9,1.006911,0.907704,0.677000
10,0.956603,0.859471,0.695800


In [39]:
# batch_size x2
bs = 512
np.random.seed(42)

In [40]:
ds_tfms = ([*rand_pad(4, 32), flip_lr(p=0.5)], [])
data = ImageDataBunch.from_folder(path, valid='test', ds_tfms=ds_tfms, bs=bs).normalize(cifar_stats)

In [41]:
learn = create_cnn(data, models.resnet34, metrics=accuracy).to_fp16()

In [42]:
with gpu_mem_restore_ctx():
    learn.fit_one_cycle(30)

epoch,train_loss,valid_loss,accuracy
1,2.207304,1.793459,0.384900
2,1.890429,1.580372,0.456000
3,1.692222,1.438321,0.499100
4,1.541481,1.340112,0.529800
5,1.422017,1.256294,0.556900
6,1.315755,1.179168,0.582300
7,1.224775,1.092550,0.612600
8,1.151535,1.041269,0.628700
9,1.091863,0.978687,0.652000
10,1.040124,0.927148,0.668900


## ResNet 50 in FP32

In [43]:
bs = 256
np.random.seed(42)

In [44]:
ds_tfms = ([*rand_pad(4, 32), flip_lr(p=0.5)], [])
data = ImageDataBunch.from_folder(path, valid='test', ds_tfms=ds_tfms, bs=bs).normalize(cifar_stats)

In [45]:
learn = create_cnn(data, models.resnet50, metrics=accuracy)

In [46]:
with gpu_mem_restore_ctx():
    learn.fit_one_cycle(30)

epoch,train_loss,valid_loss,accuracy
1,1.848420,1.540440,0.468800
2,1.620191,1.404477,0.512800
3,1.473224,1.310934,0.543500
4,1.345741,1.204520,0.577200
5,1.215567,1.085105,0.617500
6,1.099357,0.982898,0.650100
7,1.001819,0.912028,0.677700
8,0.940253,0.857122,0.697200
9,0.874114,0.786205,0.721900
10,0.808029,0.734436,0.741100


## ResNet 50 in FP16 (Mixed-Precision)

In [47]:
bs = 256
np.random.seed(42)

In [48]:
ds_tfms = ([*rand_pad(4, 32), flip_lr(p=0.5)], [])
data = ImageDataBunch.from_folder(path, valid='test', ds_tfms=ds_tfms, bs=bs).normalize(cifar_stats)

In [49]:
learn = create_cnn(data, models.resnet50, metrics=accuracy).to_fp16()

In [50]:
with gpu_mem_restore_ctx():
    learn.fit_one_cycle(30)

epoch,train_loss,valid_loss,accuracy
1,1.874705,1.542019,0.466500
2,1.634290,1.421018,0.508000
3,1.457578,1.306729,0.542100
4,1.341732,1.196863,0.579200
5,1.206769,1.083425,0.616800
6,1.095285,0.989970,0.651400
7,1.011274,0.910278,0.675000
8,0.926459,0.838621,0.710000
9,0.870166,0.775974,0.725900
10,0.823831,0.737834,0.738200


In [51]:
# batch_size x2
bs = 512
np.random.seed(42)

In [52]:
ds_tfms = ([*rand_pad(4, 32), flip_lr(p=0.5)], [])
data = ImageDataBunch.from_folder(path, valid='test', ds_tfms=ds_tfms, bs=bs).normalize(cifar_stats)

In [53]:
learn = create_cnn(data, models.resnet50, metrics=accuracy).to_fp16()

In [54]:
with gpu_mem_restore_ctx():
    learn.fit_one_cycle(30)

epoch,train_loss,valid_loss,accuracy
1,1.994531,1.599575,0.443700
2,1.721761,1.429510,0.498600
3,1.548639,1.332850,0.530900
4,1.426367,1.247172,0.564500
5,1.304639,1.135625,0.600200
6,1.199581,1.052560,0.625300
7,1.109800,0.976740,0.649700
8,1.024872,0.909431,0.672500
9,0.959838,0.856623,0.695600
10,0.898899,0.809519,0.712400


## ResNet 101 in FP32

In [55]:
bs = 256
np.random.seed(42)

In [56]:
ds_tfms = ([*rand_pad(4, 32), flip_lr(p=0.5)], [])
data = ImageDataBunch.from_folder(path, valid='test', ds_tfms=ds_tfms, bs=bs).normalize(cifar_stats)

In [57]:
learn = create_cnn(data, models.resnet101, metrics=accuracy)

In [58]:
with gpu_mem_restore_ctx():
    learn.fit_one_cycle(30)

epoch,train_loss,valid_loss,accuracy
1,1.884091,1.595758,0.451000
2,1.645583,1.447144,0.497900
3,1.467901,1.324751,0.539700
4,1.318394,1.198310,0.575700
5,1.198785,1.073593,0.622400
6,1.082813,0.992960,0.649100
7,0.986228,0.908912,0.676900
8,0.901406,0.836065,0.705100
9,0.848505,0.762788,0.732300
10,0.796432,0.715323,0.752400


## ResNet 101 in FP16 (Mixed-Precision)

In [59]:
bs = 256
np.random.seed(42)

In [60]:
ds_tfms = ([*rand_pad(4, 32), flip_lr(p=0.5)], [])
data = ImageDataBunch.from_folder(path, valid='test', ds_tfms=ds_tfms, bs=bs).normalize(cifar_stats)

In [61]:
learn = create_cnn(data, models.resnet101, metrics=accuracy).to_fp16()

In [62]:
with gpu_mem_restore_ctx():
    learn.fit_one_cycle(30)

epoch,train_loss,valid_loss,accuracy
1,1.895578,1.594412,0.448800
2,1.637513,1.458618,0.497100
3,1.474932,1.304830,0.541700
4,1.323348,1.188804,0.579300
5,1.194362,1.080506,0.624700
6,1.081963,0.984070,0.653100
7,0.989876,0.909162,0.680900
8,0.901310,0.856920,0.702800
9,0.846118,0.765747,0.734100
10,0.791107,0.721585,0.749800


In [63]:
# batch_size x2
bs = 512
np.random.seed(42)

In [64]:
ds_tfms = ([*rand_pad(4, 32), flip_lr(p=0.5)], [])
data = ImageDataBunch.from_folder(path, valid='test', ds_tfms=ds_tfms, bs=bs).normalize(cifar_stats)

In [65]:
learn = create_cnn(data, models.resnet101, metrics=accuracy).to_fp16()

In [66]:
with gpu_mem_restore_ctx():
    learn.fit_one_cycle(30)

epoch,train_loss,valid_loss,accuracy
1,2.012966,1.633735,0.431600
2,1.745837,1.484488,0.484900
3,1.567811,1.365236,0.520600
4,1.421356,1.246712,0.558700
5,1.292693,1.147737,0.590600
6,1.188780,1.059251,0.626700
7,1.088075,0.969922,0.658800
8,1.013098,0.911393,0.679600
9,0.937429,0.848215,0.704200
10,0.879763,0.807160,0.717700


## ResNet 152 in FP32

In [67]:
bs = 256
np.random.seed(42)

In [68]:
ds_tfms = ([*rand_pad(4, 32), flip_lr(p=0.5)], [])
data = ImageDataBunch.from_folder(path, valid='test', ds_tfms=ds_tfms, bs=bs).normalize(cifar_stats)

In [69]:
learn = create_cnn(data, models.resnet152, metrics=accuracy)

In [70]:
with gpu_mem_restore_ctx():
    learn.fit_one_cycle(30)

epoch,train_loss,valid_loss,accuracy
1,1.844860,1.539120,0.475400
2,1.613009,1.379866,0.518500
3,1.412254,1.245347,0.562900
4,1.277289,1.115718,0.608100
5,1.131596,0.996721,0.649000
6,1.016012,0.904986,0.676900
7,0.927867,0.813719,0.713000
8,0.844992,0.760214,0.727400
9,0.786622,0.702835,0.751100
10,0.727522,0.655739,0.769000


## ResNet 152 in FP16 ((Mixed-Precision)

In [71]:
bs = 256
np.random.seed(42)

In [72]:
ds_tfms = ([*rand_pad(4, 32), flip_lr(p=0.5)], [])
data = ImageDataBunch.from_folder(path, valid='test', ds_tfms=ds_tfms, bs=bs).normalize(cifar_stats)

In [73]:
learn = create_cnn(data, models.resnet152, metrics=accuracy).to_fp16()

In [74]:
with gpu_mem_restore_ctx():
    learn.fit_one_cycle(30)

epoch,train_loss,valid_loss,accuracy
1,1.868627,1.537747,0.467700
2,1.639547,1.385520,0.508600
3,1.422633,1.244217,0.567400
4,1.274187,1.123731,0.604200
5,1.123371,1.001790,0.640500
6,1.020521,0.917442,0.677800
7,0.926033,0.833006,0.706100
8,0.854635,0.754470,0.735900
9,0.788522,0.704508,0.753700
10,0.741587,0.658906,0.767000


In [75]:
# batch_size x2
bs = 512
np.random.seed(42)

In [76]:
ds_tfms = ([*rand_pad(4, 32), flip_lr(p=0.5)], [])
data = ImageDataBunch.from_folder(path, valid='test', ds_tfms=ds_tfms, bs=bs).normalize(cifar_stats)

In [77]:
learn = create_cnn(data, models.resnet152, metrics=accuracy).to_fp16()

In [78]:
with gpu_mem_restore_ctx():
    learn.fit_one_cycle(30)

epoch,train_loss,valid_loss,accuracy
1,1.989932,1.579180,0.458000
2,1.727004,1.427793,0.503100
3,1.534884,1.313578,0.538700
4,1.379764,1.184770,0.584000
5,1.239977,1.079134,0.619400
6,1.131927,0.988951,0.646800
7,1.036962,0.904503,0.677900
8,0.952997,0.845518,0.699500
9,0.879160,0.786672,0.721600
10,0.822652,0.746948,0.733400
